# LDA Topic modeling 
## 의료와 관련된 자료로 자세한 건 언급할 수 없다

In [17]:
library(stringr)
library(topicmodels)
library(openxlsx)
library(rJava)
library(tm)
library(dplyr)
library(NIADic)
library(LDAvis)
library(ldatuning)
library(servr)
library(KoNLP) #rJava library를 먼저 구동해야함.
library(tidyverse)
library(tidytext)

In [2]:
safe<-read.csv("C:/Users/admin/Desktop/medication_error.csv",stringsAsFactors = F)

## 특수문자 및 오타(자음,모음)들을 제거하면 용이하다

In [3]:

safe$situation_measures<-str_replace_all(safe$situation_measures,"\\("," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"\\)"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"’"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"  "," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"”"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,","," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"\\d"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"<"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,">"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"\\+"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,":"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"%"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"="," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"/"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"\\["," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"\\]"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,";"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"-"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"_"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"±"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"'"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"\\."," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"®"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"“"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"’"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"‘"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㄱ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"~"," ")

safe$situation_measures<-str_replace_all(safe$situation_measures,"\\?"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"!"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅡ"," ")

safe$situation_measures<-str_replace_all(safe$situation_measures,"ㄱ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㄴ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㄷ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㄹ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅁ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅂ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅅ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅇ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅈ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅊ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅋ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅌ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅍ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅎ"," ")

safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅏ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅑ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅗ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅛ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅠ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅣ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅢ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅜ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅕ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅓ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅔ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅖ"," ")


safe$situation_measures<-str_replace_all(safe$situation_measures,"\\》"," ㅖ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅢ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"ㅜ"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"\\÷"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"\\■"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"\\→"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"\\°"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"☆"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"★"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"♀"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"♂"," ")
safe$situation_measures<-str_replace_all(safe$situation_measures,"♡"," ")

## KoNLP패키지의 extractNoun을 이용하여 텍스트를 명사로 분리한다.
### tm패키지에서 token이 사용이 안되기 때문에(한글이라) 이렇게 진행해줘야 한다.

In [6]:

doc1<-sapply(safe$situation_measures, extractNoun,USE.NAMES = F)

doc2<-list()

for(i in 1:length(doc1)){
  doc2[[i]] <- Filter(function(x){nchar(x)>=2}, doc1[[i]])
}

doc2<-sapply(doc2, paste, collapse=" ")


## 텍스트에서 명사로 분리된것을 확인할 수 있다.

In [7]:
safe$situation_measures[1]
doc2[1]

[1] "안과 수술예정인  세 남자 환아  마취전 pre medication 약 atropine    mg 을 간호사가 안과 전공의 아이디로 로그인하여 처방함  처방하는 과정에서 atropine    mg을 atropine  mg으로 잘못 처방함  간호사가 투약카드에 적힌 용량인 atropine  mg 을 환자에게 IM 으로 주사함  주치의에게 위 상황 보고 및 환자상태 관찰함  "

[1] "수술 예정 남자 환아 마취 pre medication atropine mg 간호사 전공 아이 로그인 처방 처방 과정 atropine mg atropine mg 잘못 처방 간호사 투약 카드 용량 atropine mg 환자 IM 으로 주사 주치의 상황 환자 상태 관찰"

## 이 자료를 tm 패키지를 통해 tf-idf로 간단하게 만들 수 있다.
## 이때, 다수의 텍스트 자료들을 Corpus라고 부른다. Corpus에서의 단어들의 빈도를 확인할 수 있다.


In [8]:

doc3 <- Corpus(VectorSource(doc2)) 
tdm<-TermDocumentMatrix(doc3)

m<-as.matrix(tdm)
m<-as.data.frame(m)

mm<-data.frame(rowSums(m))
mm<-arrange(mm,desc(mm$rowSums.m.))
head(mm)

,rowSums.m.
처방,670
성분,426
환자,414
의약품,396
확인,283
밀리그램,259


## 이때, 단어들을 확인할 필요가 있는데, 같은 의미를 갖는 단어들이 다수 존재하기 때문이다.
## "복용","투여","투약"과 같이 같은 의미를 갖는 단어들을 확인하여 하나의 단어로 동의어 처리를 해주는 작업이 필요하다. 

In [ ]:

#dat1<-read.csv("situation_measures.tfm.csv",stringsAsFactors = F)
#sym1<-read.csv("situation_measures.sym.csv",stringsAsFactors = F)

#sym1<-sym1[sym1$rowSums.m.>5,]

#ind1<-sym1$sym=="삭제"

#sym1<-sym1[which(ind1==F),]
#ind2<-sym1$sym==""
#sym1$sym[ind2==T]<-sym1$word[ind2==T]

#ind3<-dat1$X %in% sym1$word
#dat1<-dat1[which(ind3==T),]


#for(k in 1:length(dat1$X)){
#  for(i in 1:length(sym1$word)){
#    
#    if(dat1$X[k]==sym1$word[i]){
#      dat1$X[k]<-sym1$sym[i]
#      
#      cat(k,i,"\n")
#    }
#  }
#}



## tm패키지에서 lda를 진행하기 전에 dtm으로 변환해야한다.
## 그 후, 토픽개수를 찾기 위해 Griffiths,2004를 기준으로 grid search를 통해 진행한다.(오래걸림)

In [10]:

dtm<-as.DocumentTermMatrix(tdm)


optimal.topics1 <- FindTopicsNumber(dtm, topics = 2:50,
                                    metrics = c("Arun2010", "Griffiths2004"),method = "Gibbs"
                                    ,control = list(seed = 77, burnin = 1000,estimate.beta = TRUE,iter=1000
                                                    ,verbose = 0, save = 0), mc.cores = 2L,verbose = TRUE,set.seed(9962))


In [11]:

k=16


control_lda<- list(estimate.beta = TRUE,alpha=50/k,
                   verbose = 0, prefix = tempfile(), save = 0, keep = 0,
                   seed = 77, nstart = 1, best = TRUE,
                   delta = 0.1,
                   iter = 1000, burnin = 1000, thin = 500)

lda<-LDA(dtm, k = k, control = control_lda,method = "Gibbs")

In [18]:

lda_doc<-tidy(lda,matrix="gamma")




lda_docs<-lda_doc%>%
  group_by(document) %>%
  arrange(document,-gamma)




lda_topic<-tidy(lda,matrix="beta")

lda_topics <- lda_topic %>%
  group_by(topic) %>%
  arrange(topic,-beta)


## 각 토픽에서 해당 단어가 뽑힐 확률 beta

In [20]:
head(lda_topics)

topic,term,beta
1,밀리그램,0.09463656
1,수출,0.07628793
1,몬테루카스트나트륨,0.02618207
1,수정,0.02124206
1,클라불란산칼륨,0.02124206
1,아목시실린,0.02053634


## 각 문서에 해당 토픽이 뽑힐 확률 gamma

In [21]:
head(lda_docs)

document,topic,gamma
1,14,0.11128049
1,3,0.09908537
1,12,0.08689024
1,16,0.08689024
1,4,0.07469512
1,13,0.07469512


# 계속 수정중